In [79]:
import os
import pandas as pd
import gspread
from datetime import datetime

In [116]:
def formatar_data(data) -> str:
    data_formatada = datetime.strftime(data, format = "%d/%m/%Y")
    return data_formatada

def formatar_hora(hora) -> str:
    hora_formatada = hora.strftime(format = "%H:%M:%S")
    return hora_formatada

In [15]:
x = pd.read_excel('order_h.XLSX')

In [17]:
_bopisloja =  x[['Centro', 'Pedido Internet - Bemol On-line', 'Data do Pedido Bemol On-line']]

In [85]:
bopisloja = _bopisloja.query('Centro > 0')
bopisloja['Data do Pedido Bemol On-line'] = bopisloja['Data do Pedido Bemol On-line'].apply(formatar_data)

/tmp/ipykernel_11329/3765529695.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bopisloja['Data do Pedido Bemol On-line'] = bopisloja['Data do Pedido Bemol On-line'].apply(formatar_data)


In [126]:
bopishora = x[['Pedido Internet - Bemol On-line','Data do Pedido Bemol On-line',
               'Hora do Pedido Bemol On-line','Caractere 1','Códido de Identificação do site']]
bopishora['Data do Pedido Bemol On-line'] = bopishora['Data do Pedido Bemol On-line'].apply(formatar_data)
bopishora['Hora do Pedido Bemol On-line'] = bopishora['Hora do Pedido Bemol On-line'].apply(formatar_hora)
bopishora.fillna('', inplace = True)

/tmp/ipykernel_11329/4205186140.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bopishora['Data do Pedido Bemol On-line'] = bopishora['Data do Pedido Bemol On-line'].apply(formatar_data)
/tmp/ipykernel_11329/4205186140.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bopishora['Hora do Pedido Bemol On-line'] = bopishora['Hora do Pedido Bemol On-line'].apply(formatar_hora)
/home/losmariio/.local/lib/python3.9/site-packages/pandas/core/frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set

In [41]:
basepedidos = x[['Pedido Internet - Bemol On-line','Data do Pedido Bemol On-line',
               'Hora do Pedido Bemol On-line','Códido de Identificação do site']]

In [130]:
basepedidos['Data do Pedido Bemol On-line'] = basepedidos['Data do Pedido Bemol On-line'].apply(formatar_data)
basepedidos['Hora do Pedido Bemol On-line'] = basepedidos['Hora do Pedido Bemol On-line'].apply(formatar_hora)

/tmp/ipykernel_11329/2569957811.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  basepedidos['Data do Pedido Bemol On-line'] = basepedidos['Data do Pedido Bemol On-line'].apply(formatar_data)
/tmp/ipykernel_11329/2569957811.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  basepedidos['Hora do Pedido Bemol On-line'] = basepedidos['Hora do Pedido Bemol On-line'].apply(formatar_hora)


In [47]:
fname = os.path.abspath("./test-auto-330418-221ec3d586fa.json")
gc = gspread.service_account(fname)
sh = gc.open("Cópia de Acompanhamento de pedidos SETEMBRO 2021")


In [49]:
sh_bopisloja = sh.worksheet('BOPIS Recebido Loja')

In [50]:
#sh_bopisloja.acell('B3').value

'Pedido Internet - Bemol On-line'

In [137]:
bopisloja_num = bopisloja.shape[0]+4
sh_bopisloja.update(f'A4:C{bopisloja_num}',bopisloja.values.tolist())

{'spreadsheetId': '1X631xStDv5gF8Hd4pMagssznGoPpErJ6rJIXhuiSN9I',
 'updatedRange': "'BOPIS Recebido Loja'!A4:C20",
 'updatedRows': 17,
 'updatedColumns': 3,
 'updatedCells': 51}

In [96]:
sh_bopishora = sh.worksheet('BOPIS por Hora')

In [125]:
bopishora_num = bopishora.shape[0]+2
sh_bopishora.update(f'A2:E{bopishora_num}',bopishora.values.tolist())

{'spreadsheetId': '1X631xStDv5gF8Hd4pMagssznGoPpErJ6rJIXhuiSN9I',
 'updatedRange': "'BOPIS por Hora'!A2:E304",
 'updatedRows': 303,
 'updatedColumns': 5,
 'updatedCells': 1515}

In [128]:
sh_basepedidos =  sh.worksheet('Base PEDIDOS h/h')

In [132]:
basepedidos_num = basepedidos.shape[0]+2
sh_basepedidos.update(f'A2:D{basepedidos_num}',basepedidos.values.tolist())

{'spreadsheetId': '1X631xStDv5gF8Hd4pMagssznGoPpErJ6rJIXhuiSN9I',
 'updatedRange': "'Base PEDIDOS h/h'!A2:D304",
 'updatedRows': 303,
 'updatedColumns': 4,
 'updatedCells': 1212}